In [1]:
from sqlalchemy import MetaData, Table, create_engine
from tqdm.notebook import tqdm

from config import db_connection_string, etherscan_api_key
from utils import get_l1_address_txs, get_zksync_address_txs, get_symbol_decimals_dict, insert_l1_txs, insert_zksync_txs

In [2]:
engine = create_engine(db_connection_string)
metadata = MetaData()
metadata.reflect(engine)
tx_table = Table('tx', metadata, autoload=True, autoload_with=engine)

## L1

In [ ]:
for row in tqdm(engine.execute("SELECT DISTINCT `address` FROM `grant`").fetchall()):
    txs = get_l1_address_txs(row.address, etherscan_api_key)
    txs = [tx for tx in txs if tx.get('tokenDecimal')!='']
    
    insert_l1_txs(txs, engine, tx_table)

## zksync

In [5]:
symbol_decimals_dict = get_symbol_decimals_dict()

In [ ]:
for row in tqdm(engine.execute("SELECT DISTINCT `address` FROM `grant`").fetchall()):
    txs = get_zksync_address_txs(row.address)
    txs = [tx for tx in txs if tx['tx']['type']=='Transfer' and isinstance(tx['tx']['token'], str)]

    insert_zksync_txs(txs, engine, tx_table, symbol_decimals_dict)

## remove duplicate

In [12]:
engine.execute("""
    CREATE TABLE tx_temp  LIKE tx;
    INSERT INTO tx_temp SELECT DISTINCT * FROM tx;
    DROP TABLE tx;
    ALTER TABLE tx_temp  RENAME TO tx;
""")